# MedGemma 1.5 - 胸部 X 光报告生成与 RadGraph F1 评估

**模型**: Google MedGemma 1.5 (4B)

**数据集**: MIMIC-CXR (233 samples)

**评估指标**: RadGraph F1 Score

**GPU**: A100 (80GB)

---

## ⚠️ 环境要求

- **Python**: **3.10 或 3.11（必需！）** Colab 默认 3.12 不兼容
- **GPU**: A100（80GB）
- **依赖**: transformers 4.47+, radgraph
- **Google Drive**: 需上传 `mimic_eval_single_image_final_233.csv`

## 🚨 使用前必读

1. **先运行 Step 0**（安装 Python 3.10）
2. **点击 Runtime → Restart Runtime**
3. **从 Step 1 开始 Run All**

---

## 流程

0. **安装 Python 3.10**（必需！）
1. 安装依赖（transformers + radgraph）
2. 挂载 Google Drive
3. 下载 MIMIC-CXR 数据集（kagglehub）
4. 对齐 233 CSV 的图片路径
5. 加载 MedGemma 1.5 模型
6. 批量生成报告（233 samples）
7. RadGraph F1 评估

## Step 0: 安装 Python 3.10（必需！）

⚠️ **重要**：Colab 默认是 Python 3.12，但某些依赖需要 3.10 或 3.11。

运行后需要 **Restart Runtime**（Runtime → Restart Runtime）。

In [1]:
import pandas as pd

csv_path = "/content/drive/MyDrive/medgamma/medgemma_reports_233.csv"
df_results = pd.read_csv(csv_path)

print(f"✅ 已加载 {len(df_results)} 条报告")
print(f"\n数据列：{df_results.columns.tolist()}")

✅ 已加载 233 条报告

数据列：['subject_id', 'View', 'Image_Path', 'Ground_Truth', 'Generated_Report']


## Step 1: 安装依赖

⚠️ **前置条件**：确保已安装 Python 3.10（Step 0）并重启了 Runtime。

In [2]:
import sys

print(f"当前 Python 版本: {sys.version}")

# 检查 Python 版本
py_major = sys.version_info.major
py_minor = sys.version_info.minor

if py_major == 3 and py_minor in [10, 11]:
    print(f"✅ Python 3.{py_minor} 符合要求")
elif py_major == 3 and py_minor == 12:
    print("\\n" + "="*70)
    print("ℹ️ 检测到 Python 3.12")
    print("\\ntransformers 5.x 和 radgraph 都支持 Python 3.12，可以直接使用！")
    print("\\n如果遇到兼容性问题，可以尝试：")
    print("1. 降级 transformers：pip install transformers==4.47.1")
    print("2. 或在 Colab 设置中选择旧版 runtime")
    print("="*70)
    print("\\n✅ 继续使用 Python 3.12（建议先尝试）")
else:
    print(f"\\n⚠️ 未知 Python 版本: {py_major}.{py_minor}")
    print("建议使用 Python 3.10-3.12")

print("\\n跳过 Python 安装，直接进入 Step 1")

当前 Python 版本: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
\n======================================================================
ℹ️ 检测到 Python 3.12
\ntransformers 5.x 和 radgraph 都支持 Python 3.12，可以直接使用！
\n如果遇到兼容性问题，可以尝试：
1. 降级 transformers：pip install transformers==4.47.1
2. 或在 Colab 设置中选择旧版 runtime
\n✅ 继续使用 Python 3.12（建议先尝试）
\n跳过 Python 安装，直接进入 Step 1


## Step 2: 挂载 Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive 已挂载")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive 已挂载


In [4]:
import sys

# 验证 Python 版本
py_version = f"{sys.version_info.major}.{sys.version_info.minor}"
print(f"当前 Python 版本: {py_version}")

if sys.version_info.major == 3 and sys.version_info.minor in [10, 11, 12]:
    print(f"✅ Python {py_version} 兼容")
else:
    print(f"\n⚠️ 警告：当前版本 {py_version} 未测试，推荐 3.10-3.12")

# 安装依赖
print("\n正在安装依赖...")
!pip install -U -q transformers
!pip install -q radgraph
print("\n✅ 依赖安装完成")

当前 Python 版本: 3.12
✅ Python 3.12 兼容

正在安装依赖...

✅ 依赖安装完成


## Step 3: 下载 MIMIC-CXR 数据集

使用 kagglehub 下载 MIMIC-CXR 数据集（~18GB）。首次运行需要 5-10 分钟。

In [5]:
import kagglehub
import os

print("🚀 开始下载 MIMIC-CXR 数据集（~18GB，首次运行需要 5-10 分钟）...")

# kagglehub 会自动缓存，第二次运行会很快
dataset_path = kagglehub.dataset_download("simhadrisadaram/mimic-cxr-dataset")

print(f"✅ 数据集下载完成！")
print(f"📂 存储路径: {dataset_path}")

# 验证文件结构
print("\n--- 文件夹结构预览 ---")
for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = '  ' * level
    print(f"{indent}{os.path.basename(root)}/")
    if level > 1: break  # 只显示前 2 层

🚀 开始下载 MIMIC-CXR 数据集（~18GB，首次运行需要 5-10 分钟）...


100%|██████████| 16.5G/16.5G [13:25<00:00, 22.0MB/s]

Extracting files...


✅ 数据集下载完成！
📂 存储路径: /root/.cache/kagglehub/datasets/simhadrisadaram/mimic-cxr-dataset/versions/2

--- 文件夹结构预览 ---
2/
  official_data_iccv_final/
    files/


## Step 4: 加载 233 CSV 并对齐路径

读取 `mimic_eval_single_image_final_233.csv`，并将 CSV 中的路径对齐到 kagglehub 下载的实际路径。

In [6]:
import pandas as pd
import os

# ✅ 使用 233 CSV
csv_path = "/content/drive/MyDrive/medgamma/mimic_eval_single_image_final_233.csv"

if not os.path.exists(csv_path):
    print(f"❌ 找不到 CSV 文件: {csv_path}")
    print("\n请确保 233 CSV 已上传到 Google Drive 的 medgamma 文件夹中！")
    raise FileNotFoundError(csv_path)

print(f"📂 读取 CSV: {csv_path}")
df = pd.read_csv(csv_path)
print(f"✅ 加载成功！共 {len(df)} 条数据")
print(f"\n列名: {list(df.columns)}")

# ==========================================
# 路径对齐函数
# ==========================================
dataset_root = f"{dataset_path}/official_data_iccv_final"

def fix_image_path(path_in_csv):
    """
    将 233 CSV 中的路径对齐到 kagglehub 下载的实际路径
    CSV 格式: /kaggle/input/mimic-cxr-dataset/official_data_iccv_final/files/...
    实际格式: {dataset_root}/files/...
    """
    if pd.isna(path_in_csv):
        return None

    path_str = str(path_in_csv).strip()

    # 提取从 files/ 开始的相对路径
    if 'files/' in path_str:
        relative_part = path_str.split('files/', 1)[1]
        full_path = os.path.join(dataset_root, 'files', relative_part)
        return full_path if os.path.exists(full_path) else None

    return None

# 应用路径修正
print("\n🔄 正在对齐图片路径...")
if 'Image_Path' in df.columns:
    df['Image_Path'] = df['Image_Path'].apply(fix_image_path)
    # 验证
    valid_count = df['Image_Path'].notna().sum()
    print(f"✅ 路径对齐完成！有效路径: {valid_count}/{len(df)}")

    # 显示第一个有效路径
    first_valid = df['Image_Path'].dropna().iloc[0]
    print(f"示例路径: {first_valid}")
    print(f"文件存在: {os.path.exists(first_valid)}")
else:
    print("❌ CSV 中没有 Image_Path 列！")

📂 读取 CSV: /content/drive/MyDrive/medgamma/mimic_eval_single_image_final_233.csv
✅ 加载成功！共 233 条数据

列名: ['subject_id', 'View', 'Image_Path', 'Ground_Truth', 'Generated_Report']

🔄 正在对齐图片路径...
✅ 路径对齐完成！有效路径: 233/233
示例路径: /root/.cache/kagglehub/datasets/simhadrisadaram/mimic-cxr-dataset/versions/2/official_data_iccv_final/files/p10/p10075925/s51010496/2d783c8a-492984b7-28aaf571-bfc30156-61ab26f6.jpg
文件存在: True


## Step 5: 加载 MedGemma 1.5 模型

In [12]:
from huggingface_hub import login
from google.colab import userdata

try:
    # 使用你已设置的 Secret 名称（zhuxirui11）
    hf_token = userdata.get('zhuxirui11')
    login(token=hf_token)
    print("✅ HuggingFace 登录成功！")
except Exception as e:
    print("❌ 登录失败！")
    print(f"错误信息: {e}")
    print("\n请确认：")
    print("1. 已在 https://huggingface.co/google/medgemma-1.5-4b-it 申请访问权限")
    print("2. 已在 Colab 左侧 🔑 Secrets 中添加 zhuxirui11")
    print("3. 已勾选 'Notebook access'（开关是蓝色的）")
    raise

✅ HuggingFace 登录成功！


In [13]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText

model_id = "google/medgemma-1.5-4b-it"

print(f"🤖 正在加载模型: {model_id}...")
print("⚠️ 首次加载需要下载 ~8GB 权重，请耐心等待...\n")

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # A100 支持 BF16
    device_map="auto",
    trust_remote_code=True
)

print(f"✅ 模型加载成功！")
print(f"Device: {model.device}")
if torch.cuda.is_available():
    mem_gb = torch.cuda.memory_allocated(0) / (1024**3)
    print(f"GPU 显存占用: {mem_gb:.2f} GB")

🤖 正在加载模型: google/medgemma-1.5-4b-it...
⚠️ 首次加载需要下载 ~8GB 权重，请耐心等待...



processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

The image processor of type `Gemma3ImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


...

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

✅ 模型加载成功！
Device: cuda:0
GPU 显存占用: 8.01 GB


## Step 6: 批量生成报告（233 samples）

对 233 张胸部 X 光图片生成放射学报告。

In [14]:
import re
from PIL import Image
from tqdm import tqdm

def generate_one_report(image_path, view_position):
    """
    为单张图片生成放射学报告
    """
    # 提示词
    prompt_text = (
        f"You are an expert radiologist. Describe this {view_position} view chest X-ray. "
        "Provide a concise report consisting of Findings and Impression. "
        "Focus on the heart, lungs, mediastinum, pleural space, and bones. "
        "Do NOT use bullet points, asterisks, or section headers. "
        "Do NOT include disclaimers or 'AI' warnings. "
        "Output pure medical text only."
    )

    try:
        pil_image = Image.open(image_path).convert("RGB")
    except Exception as e:
        return f"ERROR_IMAGE_LOAD: {e}"

    # 构建输入
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": pil_image},
                {"type": "text", "text": prompt_text}
            ]
        }
    ]

    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)

    input_len = inputs["input_ids"].shape[-1]

    # 推理
    with torch.inference_mode():
        generation = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=False
        )
        generation = generation[0][input_len:]  # 裁剪掉 prompt

    # ✅ 解码并强力清洗，去除格式标记
    raw_text = processor.decode(generation, skip_special_tokens=True)
    clean_text = raw_text.replace("Findings:", "").replace("Impression:", "")
    clean_text = clean_text.replace("**", "").replace("*", "")  # 去除 markdown
    clean_text = clean_text.replace("###", "").replace("##", "").replace("#", "")
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()

    return clean_text

# ==========================================
# 批量生成
# ==========================================
print(f"\n🚀 开始批量生成报告（共 {len(df)} 条）...\n")

results = []
skipped_count = 0

for idx, row in tqdm(df.iterrows(), total=len(df), desc="生成报告"):
    try:
        # ✅ 使用 Image_Path 列
        img_path = row.get('Image_Path')
        view = row.get('View', 'PA')

        # 验证路径
        if not img_path or not os.path.exists(img_path):
            skipped_count += 1
            continue

        # 生成报告
        generated_report = generate_one_report(img_path, view)

        # 检查错误
        if "ERROR_IMAGE_LOAD" in generated_report:
            skipped_count += 1
            continue

        # ✅ Ground_Truth 列
        gt_col = 'Ground_Truth' if 'Ground_Truth' in df.columns else 'text'
        ground_truth = str(row.get(gt_col, '')).strip()

        # 保存结果
        results.append({
            "subject_id": row.get('subject_id', idx),
            "View": view,
            "Image_Path": img_path,
            "Ground_Truth": ground_truth,
            "Generated_Report": generated_report
        })

    except Exception as e:
        print(f"\n❌ Error at index {idx}: {e}")
        skipped_count += 1
        continue

# 保存结果
df_results = pd.DataFrame(results)
output_path = "/content/drive/MyDrive/medgamma/medgemma_reports_233.csv"
df_results.to_csv(output_path, index=False)

print(f"\n✅ 报告生成完成！")
print(f"成功生成: {len(results)} 条")
print(f"跳过: {skipped_count} 条")
print(f"结果已保存至: {output_path}")

# 预览前 3 条
print("\n--- 前 3 条报告预览 ---")
for i in range(min(3, len(df_results))):
    print(f"\n[{i+1}] Subject: {df_results.iloc[i]['subject_id']}")
    print(f"Ground Truth: {df_results.iloc[i]['Ground_Truth'][:100]}...")
    print(f"Generated: {df_results.iloc[i]['Generated_Report'][:100]}...")


🚀 开始批量生成报告（共 233 条）...



生成报告: 100%|██████████| 233/233 [14:59<00:00,  3.86s/it]


✅ 报告生成完成！
成功生成: 233 条
跳过: 0 条
结果已保存至: /content/drive/MyDrive/medgamma/medgemma_reports_233.csv

--- 前 3 条报告预览 ---

[1] Subject: 10075925
Ground Truth: Mild pulmonary vascular congestion with mild to moderate interstitial pulmonary edema are new compar...
Generated: The heart is enlarged. There is mild pulmonary edema. The mediastinum is unremarkable. The pleural s...

[2] Subject: 10174198
Ground Truth: Lungs are clear without consolidation, effusion, or pneumothorax.  The cardiomediastinal silhouette ...
Generated: The heart size is normal. The mediastinal contours are unremarkable. The lungs are clear without foc...

[3] Subject: 10199765
Ground Truth: Subtle patchy opacity along the left heart border on the frontal view, not substantiated on the late...
Generated: The heart size is mildly enlarged. The mediastinal contours are unremarkable. The lungs are clear wi...


## Step 7: RadGraph F1 评估

使用 RadGraph 指标评估生成报告的质量。

**指标说明：**
- **RG_E**: Entity F1（实体匹配）
- **RG_ER**: Entity + Relation F1（实体+关系，论文常用指标）
- **RG_ER_bar**: Complete Match F1（完全匹配）

In [7]:
# ⚠️ RadGraph 兼容性修复（必须先运行这个 cell！）
# 修复 transformers 5.x 与 radgraph 的兼容性问题

from transformers import BertTokenizer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

fixed_methods = []

# 1. 添加 encode_plus 方法
if not hasattr(BertTokenizer, 'encode_plus'):
    def encode_plus_wrapper(self, text, *args, **kwargs):
        return self(text, *args, **kwargs)
    BertTokenizer.encode_plus = encode_plus_wrapper
    fixed_methods.append('encode_plus')

# 2. 添加 build_inputs_with_special_tokens 方法
if not hasattr(BertTokenizer, 'build_inputs_with_special_tokens'):
    def build_inputs_with_special_tokens_wrapper(self, token_ids_0, token_ids_1=None):
        # BERT: [CLS] + tokens_0 + [SEP] + tokens_1 + [SEP]
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        return [self.cls_token_id] + token_ids_0 + [self.sep_token_id] + token_ids_1 + [self.sep_token_id]
    BertTokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens_wrapper
    fixed_methods.append('build_inputs_with_special_tokens')

# 3. 添加 get_special_tokens_mask 方法（可能也会需要）
if not hasattr(BertTokenizer, 'get_special_tokens_mask'):
    def get_special_tokens_mask_wrapper(
        self, token_ids_0, token_ids_1=None, already_has_special_tokens=False
    ):
        if already_has_special_tokens:
            return super(BertTokenizer, self).get_special_tokens_mask(
                token_ids_0=token_ids_0,
                token_ids_1=token_ids_1,
                already_has_special_tokens=True,
            )
        if token_ids_1 is None:
            return [1] + ([0] * len(token_ids_0)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
    BertTokenizer.get_special_tokens_mask = get_special_tokens_mask_wrapper
    fixed_methods.append('get_special_tokens_mask')

if fixed_methods:
    print(f"✅ RadGraph 兼容性修复已应用: {', '.join(fixed_methods)}")
else:
    print("✅ BertTokenizer 已有所有必需方法")

✅ BertTokenizer 已有所有必需方法


In [11]:
from radgraph import F1RadGraphimport torchprint("🔍 开始 RadGraph F1 评估...")print(f"评估样本数: {len(df_results)}\n")# 准备数据refs = df_results["Ground_Truth"].tolist()hyps = df_results["Generated_Report"].tolist()# 过滤掉空报告valid_pairs = [(h, r) for h, r in zip(hyps, refs) if h and r and len(h.strip()) > 0]if len(valid_pairs) < len(hyps):    print(f"⚠️ 跳过 {len(hyps) - len(valid_pairs)} 条空报告")hyps_clean, refs_clean = zip(*valid_pairs) if valid_pairs else ([], [])# 初始化 RadGraph（使用 'all' 获取所有指标）f1radgraph = F1RadGraph(reward_level="all")# 计算 F1results = f1radgraph(hyps=list(hyps_clean), refs=list(refs_clean))# 解析返回值：results[0] = (simple, partial, complete)avg_scores = results[0]simple_f1 = float(avg_scores[0])    # RG_ER_bar (Complete Match)partial_f1 = float(avg_scores[1])   # RG_ER (Entity + Relation) ← 论文常用complete_f1 = float(avg_scores[2])  # RG_E (Entity)# 显示结果print("\n" + "="*60)print("MedGemma 1.5 原始版本 RadGraph F1 评估结果（百分制）")print("="*60)print(f"RG_E (Entity):              {complete_f1*100:.2f}")print(f"RG_ER (Entity+Relation):    {partial_f1*100:.2f}  ← 论文常用")print(f"RG_ER_bar (Complete Match): {simple_f1*100:.2f}")print("="*60)# 显示详细统计if len(results) > 1 and isinstance(results[1], tuple) and len(results[1]) >= 3:    simple_scores = results[1][0]    partial_scores = results[1][1]    complete_scores = results[1][2]    print(f"\n详细统计（{len(simple_scores)} 个样本）:")    print(f"  RG_E:       min={min(complete_scores)*100:.2f}, max={max(complete_scores)*100:.2f}, mean={sum(complete_scores)/len(complete_scores)*100:.2f}")    print(f"  RG_ER:      min={min(partial_scores)*100:.2f}, max={max(partial_scores)*100:.2f}, mean={sum(partial_scores)/len(partial_scores)*100:.2f}")    print(f"  RG_ER_bar:  min={min(simple_scores)*100:.2f}, max={max(simple_scores)*100:.2f}, mean={sum(simple_scores)/len(simple_scores)*100:.2f}")if torch.cuda.is_available():    peak_mem = torch.cuda.max_memory_allocated(0) / 1e9    print(f"\nPeak GPU 显存: {peak_mem:.2f} GB")print("\n✅ 评估完成！")

🔍 开始 RadGraph F1 评估...
评估样本数: 233

Using device: cuda:0
model_type not provided, defaulting to radgraph-xl

RadGraph F1 评估结果
RG_E (Entity):              0.2705
RG_ER (Entity+Relation):    0.3465  ← 论文常用
RG_ER_bar (Complete Match): 0.3640

详细统计（233 个样本）:
  RG_E:       min=0.0000, max=0.6667, mean=0.2705
  RG_ER:      min=0.0000, max=0.7429, mean=0.3465
  RG_ER_bar:  min=0.0000, max=0.7778, mean=0.3640

Peak GPU 显存: 3.27 GB

✅ 评估完成！


## 附录：RadGraph Patch（如遇到兼容性问题）

如果 RadGraph 报错 `encode_plus` 相关错误，运行下面的 patch：

In [ ]:
# RadGraph 兼容性修复
import os

radgraph_file = "/usr/local/lib/python3.12/dist-packages/radgraph/allennlp/data/tokenizers/pretrained_transformer_tokenizer.py"

if os.path.exists(radgraph_file):
    content = open(radgraph_file).read()
    if ".encode_plus(" in content:
        open(radgraph_file, "w").write(content.replace(".encode_plus(", "("))
        print("✅ RadGraph patch 已应用")
    else:
        print("✅ RadGraph 已是最新版本，无需 patch")
else:
    print("⚠️ RadGraph 文件未找到")